# Yolo Evaluating

Here you can evaluate and make inferences with the yolass model

In [1]:
%load_ext autoreload
%autoreload 1

from google.colab import drive
drive.mount("/content/drive")
# Direct to your assignment folder.
%cd /content/drive/MyDrive/Cartonomics/Cocass

Mounted at /content/drive
/content/drive/MyDrive/Cartonomics/Cocass


In [2]:
!pip install -r requirements.txt

!pip install comet_ml sahi fiftyone ultralytics huggingface_hub funcy opencv-python albumentations tqdm pylabel


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 3.8 MB/s eta 0:00:00
ERROR: Ignored the following versions that require a different python version: 0.6.0 Requires-Python >=3.5,<3.9; 0.6.0.1 Requires-Python >=3.5,<3.9; 0.6.1 Requires-Python >=3.5,<3.9; 0.6.1.1 Requires-Python >=3.5,<3.9; 0.6.2 Requires-Python >=3.5,<3.9; 0.6.3 Requires-Python >=3.5,<3.9; 0.6.4 Requires-Python >=3.5,<3.9; 0.6.5 Requires-Python >=3.5,<3.9; 0.6.5.1 Requires-Python >=3.5,<3.9; 0.6.5.2 Requires-Python >=3.5,<3.9; 0.6.6 Requires-Python >=3.5,<3.9; 0.7.0 Requires-Python >=3.5,<3.9
ERROR: Could not find a version that satisfies the requirement pywin32==306 (from versio

In [3]:
%cd /content/drive/MyDrive/Cartonomics/Cocass/notebooks

from utils import init_notebook, benchmarking, predictions
%aimport datasets, utils.predictions, utils.benchmarking,utils.data_cleaning
from utils.predictions import *

import sahi
from sahi import AutoDetectionModel
import fiftyone as fo
import torch

import os
from pathlib import Path

HOME = Path(os.getcwd()).parents[0]
HOME

print("torch available :",torch.cuda.is_available())
torch.cuda.empty_cache()

/content/drive/MyDrive/Cartonomics/Cocass/notebooks
torch available : True


## Model importation

### best pt 640 test

In [5]:
from ultralytics import YOLO

# Load the fine-tuned model
yolo_detection_model = YOLO(HOME/"runs/yolo/weighted_loss_2/weighted_loss_exp_288/weights/best.pt")

# Evaluate the model on the validation dataset
metrics = yolo_detection_model.val(data=HOME/"data/yolo_datasets/Detection/test/data.yaml", save=True, save_json=True)

# Print the evaluation results
print(metrics)


Ultralytics 8.3.49 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv10x summary (fused): 503 layers, 31,586,006 parameters, 0 gradients, 169.8 GFLOPs


100%|██████████| 755k/755k [00:00<00:00, 21.0MB/s]
val: Scanning /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/test/labels... 80 images, 34 backgrounds, 6 corrupt: 100%|██████████| 80/80 [00:36<00:00,  2.19it/s]

val: WARNING ⚠️ /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/test/images/3787305650.png: ignoring corrupt image/label: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (90,) + inhomogeneous part.
val: WARNING ⚠️ /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/test/images/3787708117.png: ignoring corrupt image/label: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (67,) + inhomogeneous part.
val: WARNING ⚠️ /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detection/test/images/3788258904.png: ignoring corrupt image/label: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (278,) + inhomogeneous part.
val: WARNING ⚠️ /content/drive/MyDrive/Cartonomics/Cocass/data/yolo_datasets/Detecti


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:08<00:00,  1.69s/it]


                   all         74       1076      0.611      0.342      0.427      0.189
Speed: 13.1ms preprocess, 63.7ms inference, 0.0ms loss, 0.8ms postprocess per image
Saving runs/detect/val13/predictions.json...
Results saved to runs/detect/val13
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7decd9965390>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,  

## Dataset importation

In [ ]:
# Importation of the dataset
dataset = fo.Dataset.from_dir(
    data_path=HOME/"data/yolo_datasets/Detection/valor_512/images",  # path to the dataset
    labels_path=HOME/"data/yolo_datasets/Detection/valor_512/labels",   # path to the labels/annotations file
    dataset_type=fo.types.COCODetectionDataset, # the type of dataset to import
)
dataset

# the annotations are imported as "detections" but we want to rename them as "annotations" to avoid confusion
#dataset.rename_sample_field("detections", "annotations")

 100% |█████████████████████| 0/0 [10.7ms elapsed, ? remaining, ? samples/s] 


INFO:eta.core.utils: 100% |█████████████████████| 0/0 [10.7ms elapsed, ? remaining, ? samples/s] 


Name:        2024.12.01.18.49.58
Media type:  None
Num samples: 0
Persistent:  False
Tags:        []
Sample fields:
    id:               fiftyone.core.fields.ObjectIdField
    filepath:         fiftyone.core.fields.StringField
    tags:             fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:         fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.Metadata)
    created_at:       fiftyone.core.fields.DateTimeField
    last_modified_at: fiftyone.core.fields.DateTimeField

In [ ]:
# To inspet the dataset on an external app
session = fo.launch_app(dataset, auto=False)
session.open_tab()

Session launched. Run `session.show()` to open the App in a cell output.


INFO:fiftyone.core.session.session:Session launched. Run `session.show()` to open the App in a cell output.



Welcome to

███████╗██╗███████╗████████╗██╗   ██╗ ██████╗ ███╗   ██╗███████╗
██╔════╝██║██╔════╝╚══██╔══╝╚██╗ ██╔╝██╔═══██╗████╗  ██║██╔════╝
█████╗  ██║█████╗     ██║    ╚████╔╝ ██║   ██║██╔██╗ ██║█████╗
██╔══╝  ██║██╔══╝     ██║     ╚██╔╝  ██║   ██║██║╚██╗██║██╔══╝
██║     ██║██║        ██║      ██║   ╚██████╔╝██║ ╚████║███████╗
╚═╝     ╚═╝╚═╝        ╚═╝      ╚═╝    ╚═════╝ ╚═╝  ╚═══╝╚══════╝ v1.0.2

If you're finding FiftyOne helpful, here's how you can get involved:

|
|  ⭐⭐⭐ Give the project a star on GitHub ⭐⭐⭐
|  https://github.com/voxel51/fiftyone
|
|  🚀🚀🚀 Join the FiftyOne Slack community 🚀🚀🚀
|  https://slack.voxel51.com
|



INFO:fiftyone.core.session.session:
Welcome to

███████╗██╗███████╗████████╗██╗   ██╗ ██████╗ ███╗   ██╗███████╗
██╔════╝██║██╔════╝╚══██╔══╝╚██╗ ██╔╝██╔═══██╗████╗  ██║██╔════╝
█████╗  ██║█████╗     ██║    ╚████╔╝ ██║   ██║██╔██╗ ██║█████╗
██╔══╝  ██║██╔══╝     ██║     ╚██╔╝  ██║   ██║██║╚██╗██║██╔══╝
██║     ██║██║        ██║      ██║   ╚██████╔╝██║ ╚████║███████╗
╚═╝     ╚═╝╚═╝        ╚═╝      ╚═╝    ╚═════╝ ╚═╝  ╚═══╝╚══════╝ v1.0.2

If you're finding FiftyOne helpful, here's how you can get involved:

|
|  ⭐⭐⭐ Give the project a star on GitHub ⭐⭐⭐
|  https://github.com/voxel51/fiftyone
|
|  🚀🚀🚀 Join the FiftyOne Slack community 🚀🚀🚀
|  https://slack.voxel51.com
|



<IPython.core.display.Javascript object>

If the the dataset imported is made of crops (if not skip this kernel)

In [ ]:
kwargs={"slice_height":1280, "slice_width":1280, "overlap_height_ratio":0.9, "overlap_width_ratio":0.9}
for sample in dataset.iter_samples(progress=True, autosave=True):
    yolo_results=fo_predict_simple(yolo_detection_model,sample,
        label_field="yolo_predictions",
        kwargs=kwargs)

If the dataset is only a full page

In [ ]:
from utils.predictions import fo_predict_with_slicing
#kwargs={"slice_height":1280, "slice_width":1280, "overlap_height_ratio":0.9, "overlap_width_ratio":0.9}
# We use Sliced Aided Hyper Inference to predict on the dataset
for sample in dataset.iter_samples(progress=True, autosave=True):
    yolo_results=fo_predict_with_slicing(yolo_detection_model,sample,
        label_field="yolo_predictions")
        #slice_height=640, slice_width=640, overlap_height_ratio=0.8, overlap_width_ratio=0.8)

 100% |█████████████████████| 0/0 [1.9ms elapsed, ? remaining, ? samples/s] 


INFO:eta.core.utils: 100% |█████████████████████| 0/0 [1.9ms elapsed, ? remaining, ? samples/s] 


This part as been added to export the results in COCO format in json to filter them and return them on fiftyone

In [ ]:
from utils.data_cleaning import fiftyone_extraction_remapping,filter_annotations
import fiftyone.utils.coco as fouc
import json

# YOLO predictions filtering
# ------------------------
# Export the dataset to a JSON file
gt_json = HOME/"data/coco_datasets/Cocass_aug/fraw_detailed_val.json"
#gt_json = HOME/"data/coco_datasets/tests/Cassini_009_LoC/f009_detailed_updated.json"
dataset.export(
    export_dir="results/jsons/val/yolo_predictions",
    data_path=HOME/"data/coco_datasets/Cocass_aug/images",
    dataset_type=fo.types.COCODetectionDataset,  # You can choose other formats as well
    label_field="yolo_predictions",
    classes=dataset.get_classes('annotations'),
    export_media=False  # Set to True if you want to export media files as well
)
fiftyone_extraction_remapping(HOME/"notebooks/results/jsons/val/yolo_predictions/labels.json",
                              gt_json,inplace=True)
filter_annotations(json_file_path=HOME/"notebooks/results/jsons/val/yolo_predictions/labels.json",conf_threshold=0.5,
                    height_width_ratio= 0.3,
                    second_height_width_ratio=3,
                    iou_threshold= 0.8,inside_threshold=0.2,inplace=False,keep_annotations=True,class_agnostic=False,
                    keep_abbey=True)
fouc.add_coco_labels(
    dataset,
    "yolo_predictions_filtered",
    (HOME/"notebooks/results/jsons/val/yolo_predictions/filtered_labels.json").as_posix(),
    classes=dataset.get_classes('annotations'),
)

Evaluation of the model (we use the base fiftyone function)

In [ ]:
yolo_results = dataset.evaluate_detections(
    "yolo_predictions_filtered",
    gt_field="annotations",
    eval_key="yolo_eval",
    compute_mAP=True,
)

ValueError: Dataset has no sample field 'yolo_predictions_filtered'

In [ ]:
print(" YOLO evaluation on test dataset :")
yolo_results.print_report(classes=dataset.get_classes('annotations'))
print("YOLO mAP :",yolo_results.mAP())

### Dataframes
This part can be used to export dataframes from the results (in csv, json or xlsx)

In [ ]:
from utils.benchmarking import fo_result2pd
yolo_df_9 = fo_result2pd(yolo_results,file_path="results/val/filtered_yolo_results.json",save_json=True)

## Confusion matrices
Those function takes as inputs the json exported from fiftyone. You may also use a built-in function from fiftyone but i find it hard to read and difficult to export.

In [ ]:
#I do not like the confusion matrix from fiftyone, I will use my own
from utils.benchmarking import fo_plot_confusion_matrix
plot = fo_plot_confusion_matrix(yolo_results,normalize=False)

In [ ]:
from utils.benchmarking import  build_confusion_matrix, plot_confusion_matrix
# Build the confusion matrix
confusion_matrix,categories = build_confusion_matrix(gt_json,pred_json)

In [ ]:
build_confusion_matrix(confusion_matrix,categories,normalize=True)